In [1]:
import altair as alt
import pandas as pd

ModuleNotFoundError: No module named 'altair'

# Data
## Input File Format
The input file format is identical to "Options 1: File" in UpSetR-shiny (https://github.com/hms-dbmi/UpSetR-shiny)

- Columns are `attribute 1, attribute 2, ... attribute N, set 1, set2, ..., set M` where `set` columns contain either `1` or `0`, `1` indicating the '⬤' representation in UpSet)

# Visualization
#### The UpSetAltair visualizations contain three main views: 

(1) **vertical bar chart** on the top showing the cardinality of each intersecting set;

(2) **matrix view** on the bottom-left showing the intersecting set;

~~(3) **horizontal bar chart** on the bottom-right showing the cardinality of each set.~~

# Process data

DeLUCS hit and misses are from c1normDeLUCS.png
Also, [SJSU - Ebay slides](https://docs.google.com/presentation/d/1_dlGqlGHNXgbPxo1PruNfi8C1a-fcNqzBPsIBoeHA9I/edit?usp=sharing) are being used to double check hit and misses

I am plotting mbarc genome size so slide 29 is being used


In [52]:
from enum import Enum

# better enum class name??
class Tools(Enum):
    DELUCSHIT = "DeLUCS hit"
    DELUCSMISS = "DeLUCS miss"
    PHYLOFLASHHIT = "phyloFlash hit"
    PHYLOFLASHMISS = "phyloFlash miss"

In [53]:
source = pd.read_csv("averaged-mean-coverages.csv")

df = pd.DataFrame(source, columns=['Species', 'Whole Genome', '16S'])
# new columns 1 | 0 for membership (1 = True)
# phyloFlash hit | phyloFlash miss | DeLUCS hit | DeLUCS miss

# most of phyloFlash hit so set all columns to 1 and change the few that miss
df[Tools.PHYLOFLASHHIT.value] = 1
df[Tools.PHYLOFLASHMISS.value] = 0
# most of DeLUCS hit so set all columns to 1 and change the few that miss
df[Tools.DELUCSHIT.value] = 1
df[Tools.DELUCSMISS.value] = 0

### DeLUCS miss [total 4](https://drive.google.com/file/d/1PSZ_mhcrcZkt2JlWj6S-4rmYLKxkI_PA/view?usp=sharing)

Olsenella_uli, Salmonella_bongori, Segniliparus_rotundus, Streptococcus_pyogenes

In [54]:
df.loc[df["Species"] == "Olsenella_uli", Tools.DELUCSMISS.value] = 1
df.loc[df["Species"] == "Olsenella_uli", Tools.DELUCSHIT.value] = 0

df.loc[df["Species"] == "Salmonella_bongori", Tools.DELUCSMISS.value] = 1
df.loc[df["Species"] == "Salmonella_bongori", Tools.DELUCSHIT.value] = 0

df.loc[df["Species"] == "Segniliparus_rotundus", Tools.DELUCSMISS.value] = 1
df.loc[df["Species"] == "Segniliparus_rotundus", Tools.DELUCSHIT.value] = 0


df.loc[df["Species"] == "Streptococcus_pyogenes", Tools.DELUCSMISS.value] = 1
df.loc[df["Species"] == "Streptococcus_pyogenes", Tools.DELUCSHIT.value] = 0

### phyloFlash miss [total 4](https://docs.google.com/spreadsheets/d/1bfky2TWEFXRBDVJEu07mVmbjfhVjrO01BbkltOo915A/edit#gid=939002145)

E.coli, Nocardiopsis_dassonvillei, Salmonella_bongori, Fervidobacterium_pennivorans

In [55]:
df.loc[df["Species"] == "Nocardiopsis_dassonvillei", Tools.PHYLOFLASHMISS.value] = 1
df.loc[df["Species"] == "Nocardiopsis_dassonvillei", Tools.PHYLOFLASHHIT.value] = 0

df.loc[df["Species"] == "E.coli", Tools.PHYLOFLASHMISS.value] = 1
df.loc[df["Species"] == "E.coli", Tools.PHYLOFLASHHIT.value] = 0

df.loc[df["Species"] == "Salmonella_bongori", Tools.PHYLOFLASHMISS.value] = 1
df.loc[df["Species"] == "Salmonella_bongori", Tools.PHYLOFLASHHIT.value] = 0

df.loc[df["Species"] == "Fervidobacterium_pennivorans", Tools.PHYLOFLASHMISS.value] = 1
df.loc[df["Species"] == "Fervidobacterium_pennivorans", Tools.PHYLOFLASHHIT.value] = 0

# Custom upset plot

In [56]:
width=2000
height=800
height_ratio=0.6
# currently no horizontal bar graph
horizontal_bar_chart_width=300

# Species labels size
bottom_x_axis_labels_size = 13
bottom_x_axis_labels_weight = alt.FontWeight("normal")

# bar chart settings
vertical_bar_label_size=14
vertical_bar_title_size=23
vertical_bar_thickness=40

# matrix view settings
glyph_size = 200
matrix_label_size = 12
dot_line_connection_size=2

# on mouse hover tooltip show Genome size value
tooltip = [
    alt.Tooltip("Whole Genome", title="Size"),
]

vertical_bar_chart_height = height * height_ratio
matrix_height = (height - vertical_bar_chart_height) * 0.4
matrix_width = (width - horizontal_bar_chart_width)

main_color = "#303030"
highlight_color = "#09c372"

# on mouse hover over Species
mouse_hover_selection = alt.selection_multi(on="mouseover", fields=["Species"])
# applies highlight on mouse hover of Species
# main_color when no mouse hover
#  note the ~
brush_color = alt.condition(~mouse_hover_selection, alt.value(main_color), alt.value(highlight_color))


##### end tuning parameters



# sorting by Whole Genome Descending to be used by upset plot
df = df.sort_values(by="Whole Genome", ascending=False)
# no dot in a hit row = tool misses
df = df.drop([Tools.PHYLOFLASHMISS.value, Tools.DELUCSMISS.value], axis=1)
upset_plot_sorting = alt.SortField(field='Whole Genome', order='descending')

In [57]:
# vertical bar chart
vertical_bar = alt.Chart(df.copy(), title='Genome Size').mark_bar(color=main_color, size=vertical_bar_thickness).encode(
    x=alt.X(
        "Species", 
        axis=alt.Axis(grid=False, labels=False, ticks=False, domain=True),
        title=None,
        sort=upset_plot_sorting,
    ),
    y=alt.Y(
        "Whole Genome",
        axis=alt.Axis(grid=False, orient='right', values=[0, 10000 ,1000000, 6000000]),
        title=None,
        scale=alt.Scale(type="log")
        # scale=alt.Scale(type="log", base=10000)
    ),
    tooltip=tooltip,
    color=brush_color
).properties(
    width=matrix_width,
    height=vertical_bar_chart_height
)

vertical_bar.configure_title(fontSize=10000)

vertical_bar_text = vertical_bar.mark_text(
    color=main_color, 
    dy=-10,
    size=vertical_bar_label_size,
    fontWeight='bold',
).encode(
    text=alt.Text("Whole Genome")
)

vertical_bar_line = vertical_bar.mark_line(color=highlight_color, opacity=1).encode(
    color=alt.value(highlight_color),
    y=alt.Y(
            '16S:Q',
           )
    )

line_text = vertical_bar_line.mark_text(
    color=main_color,
    size=vertical_bar_label_size,
    align='center',
    baseline='middle',
    dy=-18,
    fontWeight='bold',
    fontSize=13,
).encode(
    text=alt.Text("16S:Q")
)

bardata = pd.DataFrame({
    'y':['16S'],
    'x':['Genome Size']
                       
})

legendline = alt.Chart(bardata).mark_tick(
        size=15, 
        thickness=3, 
        orient='horizontal', 
        color='black',
        opacity=1
).encode(
        color=alt.value(highlight_color),
            y=alt.Y('y:N',
                    axis=alt.Axis(orient='right', 
                                  titleFontSize=0, 
                                  labelFontSize=16, 
                                  titleX=80, 
                                  titleY=-5,
                                  titleAngle=0
                                 ),

                   ),
    )

legendbar = alt.Chart(bardata).mark_square(size=200, color="#303030").encode(
    y=alt.Y('x:N', 
            axis=alt.Axis(orient='right', 
                          titleFontSize=0, 
                          labelFontSize=16, 
                          titleX=80, 
                          titleY=-5,
                          titleAngle=0
                         ),
    
           ),
    )


# Combines the separate components to create the vertical bar chart
# mouse interaction added
vertical_bar_chart = (vertical_bar + vertical_bar_text + vertical_bar_line + line_text).add_selection(
    mouse_hover_selection
)


In [58]:
# matrix view selections
selection2 = alt.selection_multi(on="mouseover", fields=["Species", "value"])
circle_color = alt.condition(~selection2, alt.Color("value:N", scale=alt.Scale(domain=[0,1], range=["#E6E6E6", main_color]), legend=None), alt.value(highlight_color))

In [59]:
# UpSet glyph view (matrix view) of hits
# value of 0 = miss
matrix_base = alt.Chart(df.copy()).mark_circle(
    size=glyph_size,
    opacity=1
).transform_fold(
    [Tools.PHYLOFLASHHIT.value, Tools.DELUCSHIT.value]
).encode(
    x=alt.X(
        "Species",
        axis=alt.Axis(grid=False, ticks=False, domain=False, labelFontSize=bottom_x_axis_labels_size, labelFontWeight=bottom_x_axis_labels_weight),
        title=None,
        sort=upset_plot_sorting
    ),
    y=alt.Y(
        "key:N",
        axis=alt.Axis(grid=False, ticks=False, domain=False, orient='right', labelFontWeight="bold"),
        title=None
    ),
    color=circle_color,
).properties(
    height=matrix_height,
    width= matrix_width
)

# keeps miss dots from being highlighted on mouse hover
grey_glyph_size = glyph_size + 30
circle_grey_miss = matrix_base.mark_circle(size=grey_glyph_size, opacity=1).transform_filter(
    (alt.datum["value"] == 0)
).encode(
    color=alt.value("#E6E6E6")
)

# connect dots new changes might of broken but might no longer be needed
# line_connect_dots = matrix_base.mark_bar(size=dot_line_connection_size, color=main_color).transform_filter(
#     alt.datum["value"] == 1
# ).encode(
#     y=alt.Y("min(key):N"),
#     y2=alt.Y2("max(key):N"),
#     color=alt.value(main_color)
# )

# highlights every other row of matrix view to distinguish hit and miss rows 
zebra_strip_matrix_view = matrix_base.mark_rect().transform_filter(
    (alt.datum.key == "DeLUCS hit")
).encode(
    color=alt.value("#F7F7F7")
)

# Combines the separate components to create the matrix view
# order seems matter
# mouse interaction added
# Duplicate `circle` is to properly show tooltips and on mouse color highlighting
matrix_view = (matrix_base + zebra_strip_matrix_view + matrix_base + circle_grey_miss).add_selection(
    mouse_hover_selection,
    selection2
)




In [60]:
# combine all charts to make final upset plot
upset_plot = alt.hconcat(
    vertical_bar_chart,
    legendline + legendbar
)

upset_plot = alt.vconcat(
    upset_plot,
    matrix_view
)

# final graph settings tuning
upset_plot.configure_view(
    stroke=None
).configure_axis(
    titleFontSize=vertical_bar_title_size,
    labelFontSize=matrix_label_size
).configure_title(
    fontSize=25
)


alt.VConcatChart(...)